In [9]:
from pylogic import *
from pylogic.proposition.proof_search import proof_search, _BackwardProver

In [10]:
# Example usage
A = propositions("A", is_assumption=True)
B, C, D, E, F, G = propositions("B", "C", "D", "E", "F", "G")
p6 = Implies(And(Or(D, E), Or(A, F)), G, is_assumption=True)
A_imp_B = Implies(A, B, is_assumption=True)
B_imp_C = Implies(B, C, is_assumption=True)
C_imp_D = Implies(C, D, is_assumption=True)
kb = [
    # A,
    # A_imp_B,
    # B_imp_C,
    # C_imp_D,
    C.implies(G).assume(),
    B.implies(F).assume(),
    B.or_(C).assume(),
    # p6,
]
target = F.or_(G)
proof = proof_search(kb, target)
print(proof)


F \/ G


In [11]:
P, Q, R, S, T = propositions("P", "Q", "R", "S", "T")
p1 = P.implies(Q).and_(R.implies(S)).assume()
p2 = Q.or_(S).implies(T).assume()
p3 = P.or_(R).assume()
p4 = neg(T).assume()

target = neg(P).and_(neg(R))
kb = [
    p1,
    p2,
    p3,
    p4,
]
proof = proof_search(kb, target)
print(proof.deduced_from)

('ex_falso', Proposition(contradiction))


In [12]:
A, B = propositions("A", "B")
p1 = A.or_(B).assume()
p2 = neg(B).assume()

target = A
kb = [
    p1,
    p2,
]
proof = proof_search(kb, target)
print(proof.deduced_from)

('by_cases', Or(Proposition(A), Proposition(B)), 'has inner contexts')


In [13]:
A, B, C, D, E, F = propositions("A", "B", "C", "D", "E", "F")
settings["USE_CLASSICAL_LOGIC"] = True

# Premises
p1 = A.implies(B.and_(F)).assume()
p2 = D.implies(neg(B)).assume()
p3 = C.or_(D).assume()
p4 = neg(E).implies(neg(D)).assume()

# Target
target = A.implies(C)

kb = [
    p1,
    p2,
    p3,
    # p4,
]

proof = proof_search(kb, target)
print(proof.deduced_from)


('by_cases', Or(Proposition(C), Proposition(D)), 'has inner contexts')


In [14]:
I, J, K, M, N = propositions("I", "J", "K", "M", "N")
p1 = I.implies(J).assume()
p2 = J.or_(I, K).assume()
p3 = J.implies(M).assume()
p4 = neg(M).assume()
p5 = K.implies(neg(N)).assume()

target = neg(N)
kb = [
    p1,
    p2,
    p3,
    p4,
    p5,
]

In [15]:
_BackwardProver(kb)

In [16]:

proof = proof_search(kb, target)
print(proof.deduced_from)

('modus_ponens', Proposition(K), Implies(Proposition(K), Not(Proposition(N))))


In [17]:
from collections import namedtuple
from random import choice

card = namedtuple("card", ["letter", "color"])
letters = ["A",  "B", "E", "K"]; vowels = ["A", "E"]
colors = ["blue", "red"]

face_up = ["E", "K", "blue", "red"]
cards = []
for c in face_up:
    if c in colors:
        cards.append(card(choice(letters), c))
    else:
        cards.append(card(c, choice(colors)))

card_terms = [Constant(f"card_{i}") for i in range(len(cards))]
is_vowel, is_blue = predicates("is_vowel", "is_blue")

kb = []
targets = [is_vowel(card_terms[i]).implies(is_blue(card_terms[i])) for i in range(len(cards))]
for i,c in enumerate(cards):
    if c.letter in vowels:
        kb.append(is_vowel(card_terms[i]).assume())
    else:
        kb.append(neg(is_vowel(card_terms[i])).assume())
    if c.color == "blue":
        kb.append(is_blue(card_terms[i]).assume())
    else:
        kb.append(neg(is_blue(card_terms[i])).assume())


for i,t in enumerate(targets):
    try:
        proof = proof_search(kb, t)
        print(f"Card {i} satisfies the condition")
    except ValueError:
        print(f"Card {i} does not satisfy the condition")


Card 0 does not satisfy the condition
Card 1 satisfies the condition
Card 2 satisfies the condition
Card 3 does not satisfy the condition


In [18]:
A, B = propositions("A", "B")
kb = []

target = B.implies(neg(A).implies(B)).assume()

proof = proof_search(kb, target)
print(proof.deduced_from)

('close_assumptions_context', 'has inner contexts')


In [19]:

P, Q, R, S = propositions("P", "Q", "R", "S")

p1 = neg(P.and_(Q.or_(R, S)))#.assume()
p2 = neg(P).or_(neg(Q).and_(neg(R), neg(S))).assume()

kb = [
    p2,
]

target = p1
proof = proof_search(kb, target)
print(proof.deduced_from)

('by_cases', Or(Not(Proposition(P)), And(Not(Proposition(Q)), Not(Proposition(R)), Not(Proposition(S)))), 'has inner contexts')


In [20]:
P, Q, R, S = propositions("P", "Q", "R", "S")
p1 = neg(P.and_(Q)).assume()
p2 = neg(P).or_(neg(Q))#.assume()
kb = [
    p1,
]
target = p2
proof = proof_search(kb, target)
print(proof.deduced_from)


ValueError: No rule found to prove ~P \/ ~Q

In [21]:
P, Q, R, S = propositions("P", "Q", "R", "S")
p1 = neg(P.and_(Q))#.assume()
p2 = neg(P).or_(neg(Q)).assume()
kb = [
    p2,
]
target = p1
proof = proof_search(kb, target)
print(proof.deduced_from)


('by_cases', Or(Not(Proposition(P)), Not(Proposition(Q))), 'has inner contexts')


In [22]:
P, Q, R, S = propositions("P", "Q", "R", "S")

p1 = P.assume()
kb = [
    p1,
]
target = neg(neg(P))
proof = proof_search(kb, target)
print(proof.deduced_from)

None


In [23]:
from pylogic.enviroment_settings.settings import settings

settings["USE_CLASSICAL_LOGIC"] = False
P, Q, R, S = propositions("P", "Q", "R", "S")

p1 = neg(neg(P)).assume()
kb = [
    p1,
]
target = P
proof = proof_search(kb, target)
print(proof.deduced_from)

ValueError: No rule found to prove P